In [1]:
from model import efficientdet
import cv2
import glob
import os
import numpy as np
import time
from utils import preprocess_image
from utils.anchors import anchors_for_shape
from utils.draw_boxes import draw_boxes
from utils.post_process_boxes import post_process_boxes


def main():
    os.environ['CUDA_VISIBLE_DEVICES'] = '1'
    
    phi = 1
    weighted_bifpn = False
    model_path = 'pascal_05_0.6283_1.1975_0.8029.h5'
    image_sizes = (512, 640, 768, 896, 1024, 1280, 1408)
    image_size = image_sizes[phi]
    classes = ['aeroplane', 'bicycle', 'bird', 'boat', 'bottle', 'bus', 'car', 'cat', 'chair', 'cow', 'diningtable',
               'dog', 'horse', 'motorbike', 'person', 'pottedplant', 'sheep', 'sofa', 'train', 'tvmonitor']
    num_classes = len(classes)
    score_threshold = 0.5
    colors = [np.random.randint(0, 256, 3).tolist() for _ in range(num_classes)]
    model, prediction_model = efficientdet(phi=phi,
                                           weighted_bifpn=weighted_bifpn,
                                           num_classes=num_classes,
                                           score_threshold=score_threshold)
    prediction_model.load_weights(model_path, by_name=True)
    
  
    image_file = glob.glob('*.jpg')
    for img_file in image_file:
        image = cv2.imread(img_file)
        src_image = image.copy()
        image = image[:,:,::-1]
        h,w = image.shape[:2]
        
        image, scale, offset_h, offset_w  = preprocess_image(image, image_size=image_size)
        anchors = anchors_for_shape((image_size, image_size))
    
    # run network
        start = time.time()
        boxes, scores, labels = prediction_model.predict_on_batch([np.expand_dims(image, axis=0),
                                                               np.expand_dims(anchors, axis=0)])
        boxes, scores, labels = np.squeeze(boxes), np.squeeze(scores), np.squeeze(labels)
        print(time.time() - start)
        boxes = post_process_boxes(boxes=boxes,
                               scale=scale,
                               offset_h=offset_h,
                               offset_w=offset_w,
                               height=h,
                               width=w)
    
    # select indices which have a score above the threshold
        indices = np.where(scores[:] > score_threshold)[0]
    
    # select those detections
        boxes = boxes[indices]
        labels = labels[indices]
    
        draw_boxes(src_image, boxes, scores, labels, colors, classes)
    
        cv2.namedWindow('image', cv2.WINDOW_NORMAL)
        cv2.imshow('image', src_image)
        cv2.waitKey(1)
    #cv2.destroyAllWindows()


if __name__ == '__main__':
    main()


Instructions for updating:
`normal` is a deprecated alias for `truncated_normal`
Instructions for updating:
If using Keras pass *_constraint arguments to layers.

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
3.271655321121216
0.5124175548553467
0.43779778480529785
0.4298527240753174
0.43779778480529785
0.4268608093261719
0.4438135623931885
0.4593770503997803
0.438823938369751
0.43460702896118164
0.4348330497741699
0.4340505599975586
0.4411153793334961
0.4586172103881836
0.43779540061950684
0.4438142776489258
0.44377684593200684
